In [1]:
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [229]:
############################ INPUT ################

Par_rates = pd.read_excel('Mod-MIF PAR.xlsx')
SOFR_Zero = pd.read_excel('SOFR Zero.xlsx')
calendar = pd.read_excel('INR_Calendar.xlsx')
FX_Spot = 82.31

###################################################

In [185]:
cal = calendar
cal = [str(i) for i in cal['Working_Days']]

In [186]:
def check_calendar(x):
    x = str(x)
    y = x in cal
    return(y)

In [187]:
Tenors = Par_rates.columns.values
Tenors

array(['Dates', 'ON', 'TN', 'SN', '1W', '1M', '2M', '3M', '4M', '5M',
       '6M', '7M', '8M', '9M', '10M', '11M', '12M', '15M', '18M', '21M',
       '2Y', '3Y', '4Y', '5Y', '7Y', '10Y'], dtype=object)

In [269]:
def make_tenors(k):
    
    end_date = []
    spot_date = Par_rates['Dates'].iloc[k-2]
    
    for i in range(1,26):
        
        if(Tenors[i][len(Tenors[i])-1] == 'N'):
            t = Par_rates['Dates'][k-i]
            end_date.append(t)
            
        elif(Tenors[i][len(Tenors[i])-1] == 'W'):
            t = spot_date + relativedelta(days = int(Tenors[i][0:len(Tenors[i])-1])*7)
            if(check_calendar(t) == True):
                end_date.append(t)
            while(check_calendar(t) == False):
                t = t + relativedelta(days = 1)
                if(check_calendar(t) == True):
                    end_date.append(t)
                    
        elif(Tenors[i][len(Tenors[i])-1] == 'M'):
            t = spot_date + relativedelta(months = int(Tenors[i][0:len(Tenors[i])-1]))
            if(check_calendar(t) == True):
                end_date.append(t)
            while(check_calendar(t) == False):
                t = t + relativedelta(days = 1)
                if(check_calendar(t) == True):
                    end_date.append(t)            
    
        elif(Tenors[i][len(Tenors[i])-1] == 'Y'):
            t = spot_date + relativedelta(years = int(Tenors[i][0:(len(Tenors[i])-1)]))
            if(check_calendar(t) == True):
                end_date.append(t)
            while(check_calendar(t) == False):
                t = t + relativedelta(days = 1)
                if(check_calendar(t) == True):
                    end_date.append(t)   
                    
    return(end_date, spot_date)

    
    

In [280]:
def search_buisness_days(k):
    
    ending_dates = []
    
    end_date, spot_date = make_tenors(k)
    end_date = end_date[19:25]
    start_dates = [spot_date]
    if(check_calendar(start_dates[0]) == False): start_dates[0] = start_dates[0] + relativedelta(days = 1)
        
    for i in range(1,20):
        start_dates.append(spot_date + relativedelta(months = i*6))
        while(check_calendar(start_dates[i]) == False):
                start_dates[i] = start_dates[i] + relativedelta(days = 1)                
        ending_dates.append(start_dates[i])
   
    ending_dates.append(end_date[len(end_date)-1])
    
    return(start_dates, ending_dates)
        

In [281]:
def ZCYC_Premia(k):
    
    End_Dates = make_tenors(k)[0]
    start = Par_rates['Dates'].iloc[k]
    cash_rate = FX_Spot - (Par_rates['ON'].iloc[k] + Par_rates['TN'].iloc[k])/100
    sofr_zero = SOFR_Zero.iloc[:,3]
    end_dates_sofr = SOFR_Zero.iloc[:,1]
    
    tenor_frac = []
    for m in range(0, len(end_dates_sofr)):
        y3 = (end_dates_sofr[m] - start).days
        tenor_frac.append(y3/365)
        
    sofr_interp = []
    fwd_rate = []
    cash_fwd = []
    df_sofr = []
    parity = []
    yf = []
    bootstrap_zero = []
    
    for i in range(0,19):
        dayz = End_Dates[i] - start
        dayz = dayz.days
        year_frac = dayz/365
        yf.append(year_frac)
        sofr_interp.append(np.interp(year_frac, list(tenor_frac), list(sofr_zero) ))
        df_sofr.append(math.exp(-sofr_interp[i]*yf[i]))
        
        if(i == 0):
            fwd_rate.append(FX_Spot - Par_rates['TN'][k]/100)
            cash_fwd.append(cash_rate/fwd_rate[i])
            parity.append(df_sofr[i]*cash_fwd[i])
            bootstrap_zero.append(parity[i]**(-1/yf[i])-1)
            
        elif(i == 1):
            fwd_rate.append(FX_Spot)
            cash_fwd.append(cash_rate/fwd_rate[i])
            parity.append(df_sofr[i]*cash_fwd[i])
            bootstrap_zero.append(parity[i]**(-1/yf[i])-1)
            
        else:
            fwd_rate.append(FX_Spot + Par_rates.iloc[k,i+1]/100)
            cash_fwd.append(cash_rate/fwd_rate[i])
            parity.append(df_sofr[i]*cash_fwd[i])
            bootstrap_zero.append(parity[i]**(-1/yf[i])-1)
            
    DF = pd.concat([pd.DataFrame(End_Dates),pd.DataFrame(bootstrap_zero)],axis = 1)
    DF.columns = ["Tenor", "ZCYC"]
    
    return(DF)

In [283]:
def NPV(k, zero_rate, ZCYC, end, start, tenor_end, par_rate, g):
    if(g == 19):
        t = 4
    elif(g == 20):
        t = 6    
    elif(g == 21):
        t = 8    
    elif(g == 22):
        t = 10    
    elif(g == 23):
        t = 14    
    elif(g == 24):
        t = 20 
        
    start = start[0:t]
    end = end[0:(t+1)]
    
    zero_rates = ZCYC
    zero_rates['ZCYC'][g] = zero_rate
    
    fix_leg = []
    float_leg = []
    mod_mif_interp = []
    mod_mif_interp_end = []
    mod_mif_interp_start = []
    df_mif = []
    df_mif_fra_start = []
    df_mif_fra_end = []
    fra = []
    yf = []
    yf_disc = []
    cash_date = Par_rates['Dates'].iloc[k]
    
    tenor_frac = []
    for m in range(0,len(zero_rates)):
        y3 = (zero_rates.iloc[m,0] - cash_date).days
        tenor_frac.append(y3/365)
        
    for i in range(0,len(start)):
        
        dayz1 = end[i] - cash_date
        dayz1 = dayz1.days
        year_frac_disc = dayz1/365
        yf_disc.append(year_frac_disc)
        mod_mif_interp.append(np.interp(yf_disc[i], list(tenor_frac), list(zero_rates['ZCYC'])))
        df_mif.append(1/(1+mod_mif_interp[i])**year_frac_disc)
        
        cf = 1e7*par_rate*((end[i] - start[i]).days/365)*df_mif[i]
        fix_leg.append(cf)
        
        dayz2 = end[i] - cash_date
        dayz2 = dayz2.days
        year_frac = dayz2/365
        mod_mif_interp_end.append(np.interp(year_frac, list(tenor_frac), list(zero_rates['ZCYC'])))
        df_mif_fra_end.append(1/(1+mod_mif_interp_end[i])**year_frac)
        
        dayz3 = start[i] - cash_date
        dayz3 = dayz3.days
        year_frac = dayz3/365
        mod_mif_interp_start.append(np.interp(year_frac, list(tenor_frac), list(zero_rates['ZCYC'])))
        df_mif_fra_start.append(1/(1+mod_mif_interp_start[i])**year_frac)
        
        fra_cf = (df_mif_fra_start[i]/df_mif_fra_end[i] -1)*365/((end[i]-start[i]).days)
        fra.append(fra_cf)
        cf = 1e7*fra[i]*((end[i]-start[i]).days/365)*df_mif[i]
        float_leg.append(cf)
        v = sum(float_leg) - sum(fix_leg)
    
    return(v)
        

In [284]:
k = 3
#ZCYC = ZCYC_Premia(k)
zero_rate = 0.07
start, end = search_buisness_days(k)
tenor_end = make_tenors(k)[0]
g = 24
NPV(k, zero_rate, ZCYC, end, start, tenor_end, Par_rates.iloc[k,(g+1)], g)
    

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


-59957.42294688802

In [285]:
def newton_raphson_full(k, zero_rate, ZCYC, end, start, tenor_end, par_rate, g):
    h = 0.00001
    f2 = NPV(k, zero_rate, ZCYC, end, start, tenor_end, par_rate, g)
    for i in range(0,10):
        f1 = NPV(k, zero_rate + h, ZCYC, end, start, tenor_end, par_rate, g)
        f2 = NPV(k, zero_rate, ZCYC, end, start, tenor_end, par_rate, g)
        df = (f1 - f2)/h
        zero_rate = zero_rate - f2/df
    
    return(zero_rate)

In [288]:
k = 3
ZCYC = ZCYC_Premia(k)
zero_rate = 0.1
start, end = search_buisness_days(k)
tenor_end = make_tenors(k)[0]

for g in range(19,25):
    print(g)
    ZCYC['ZCYC'][g] = newton_raphson_full(k, zero_rate, ZCYC, end, start, tenor_end, Par_rates.iloc[k,(g+1)], g)
    

19
20


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


21
22
23
24


In [289]:
ZCYC = pd.concat([pd.DataFrame(Tenors[1:]),ZCYC],axis = 1)
ZCYC

,0,Tenor,ZCYC
0,ON,2023-08-07,0.063719
1,TN,2023-08-08,0.081557
2,SN,2023-08-09,0.120190
3,1W,2023-08-15,0.101009
4,1M,2023-09-08,0.088139
5,2M,2023-10-08,0.088770
6,3M,2023-11-08,0.088479
7,4M,2023-12-08,0.088553
8,5M,2024-01-08,0.084607
9,6M,2024-02-08,0.081943
